In [1]:
import pandas as pd
from openai import OpenAI
import glob
import os
from wandb.sdk.data_types.trace_tree import Trace
import wandb
import configparser
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
import sys
import os
import inspect
import json
import random
# access parent directory from notebooks directory
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
config = configparser.ConfigParser()
# Read the configuration file
config.read('config.ini')
api_key_openai = config.get('credentials', 'api_key_openai')
api_key_mistral = config.get('credentials', 'api_key_mistral')
surfdrive_url_input_sentences = config.get('credentials', 'surfdrive_url_input_sentences')
surfdrive_url_prompts = config.get('credentials', 'surfdrive_url_prompts')
output_parallel_data = 'output_parallel_data/'
output_llm_folder_path = 'output_llm_data/'

### Create the context string